In [15]:
import wget
import pandas as pd
import wikipedia as wp
import numpy as np
import lxml

In [16]:
# Get the table from wiki page
html = wp.page("List of postal codes of Canada: M").html().encode("UTF-8")
df = pd.read_html(html)[1]
df = pd.read_html(html)[0]
print(df.to_string())

# Drope the rows with Borough of "Not assigned"
df_assigned = df[df.Borough != 'Not assigned']
df_assigned.reset_index(drop = True, inplace = True)

# Merge rows with the same Postcode
df_assigned = df_assigned.groupby(['Postal Code', 'Borough'], as_index=False).agg(lambda x: ', '.join(set(x)))

# Set 'Not assigned' Neigbourhood to be equal to Borough
na_nb_idx =df_assigned['Neighbourhood'] == 'Not assigned'
df_assigned.loc[na_nb_idx, 'Neighbourhood'] = df_assigned[na_nb_idx].Borough
df_assigned[df_assigned['Postal Code'] == 'M7A']

    Postal Code           Borough                                                                                                                           Neighbourhood
0           M1A      Not assigned                                                                                                                            Not assigned
1           M2A      Not assigned                                                                                                                            Not assigned
2           M3A        North York                                                                                                                               Parkwoods
3           M4A        North York                                                                                                                        Victoria Village
4           M5A  Downtown Toronto                                                                                                               Regent

,Postal Code,Borough,Neighbourhood
85,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [17]:
# Show the shape of the df
df_assigned.shape

(103, 3)

In [18]:
# Since it is not able to obtain coordinates from using geocoder.google, the given csv file is used
url = 'http://cocl.us/Geospatial_data/'
wget.download(url, '/users/User/PycharmProjects/pythonProject/Geospatial_Coordinates.csv')

'/users/User/PycharmProjects/pythonProject/Geospatial_Coordinates (1).csv'

In [19]:
df_cord = pd.read_csv('Geospatial_Coordinates.csv') # Read the csv data
df_cord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [21]:
# Create Latitude and Longitude columns in df_assigned
df_assigned['Latitude'] = np.nan
df_assigned['Longitude'] = np.nan

# For each postcode in df_assigned, find corresponding coordinates in df_cord and assign it to df_assigned
for idx in df_assigned.index:
    cord_idx = df_cord['Postal Code'] == df_assigned.loc[idx, 'Postal Code']
    df_assigned.at[idx, 'Latitude'] = df_cord.loc[cord_idx, 'Latitude'].values
    df_assigned.at[idx, 'Longitude'] = df_cord.loc[cord_idx, 'Longitude'].values

# Display the results
df_assigned.head(20)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
